# Network Update 2023

## Whats new? 

Its that time of the year again, lets see how many things needs to be changed this time.

In [2]:
from psycopg2 import connect
from psycopg2 import sql
from psycopg2.extras import execute_values
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
import configparser
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)
connection_engine = create_engine(
    URL.create(
        drivername = "postgresql",
        host = CONFIG['DBSETTINGS']['host'],
        database = CONFIG['DBSETTINGS']['database'],
        username = CONFIG['DBSETTINGS']['user'],
        password = CONFIG['DBSETTINGS']['password']))
cur = connection_engine.raw_connection().cursor()

### Number of nodes that needs to be updated
only two!

In [29]:
sql = '''
select node_id from congestion.network_nodes
except 
select node_id from here.routing_nodes_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      node_id
0  30420390.0
1  30420392.0


### Number of links that needs to be updated
2990 links needs to be retired omg

In [30]:
sql = '''
select link_dir from congestion.network_links_22_2
except 
select link_dir from here.routing_streets_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

         link_dir
0     1207662665F
1     1258933853T
2     1000005503T
3      995198819F
4       29568114T
...           ...
2985   949289551T
2986  1258735603F
2987  1258732905T
2988   993964936T
2989  1258776870T

[2990 rows x 1 columns]


### Number of segments that needs to be updated
1812 segment_ids needs be to retired and route............

In [31]:
sql = '''
with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)
	
select distinct segment_id from  congestion.network_links_22_2 
inner join changed_links using (link_dir)
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      segment_id
0           6062
1           2335
2            652
3           5856
4           1560
...          ...
1807        2359
1808        3795
1809        1715
1810        3697
1811        4035

[1812 rows x 1 columns]


### Number of new and retired traffic signals
Quite a bunch of new traffic signals. We can deal with this after we recreated the network. 

In [32]:
sql = '''
select count(1) from gis.traffic_signal
where activationdate  >= '2022-01-01'
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()   

,count
0,43


# Updating the links

Create new network_links_23_4 table with newly routed segments for ones that contains outdated links

In [ ]:
'''
-- Create new links table except the segment_ids that are outdated

CREATE TABLE congestion.network_links_23_4 AS 
-- retired links
with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)
-- retired segments	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_22_2
	inner join changed_links using (link_dir))
-- everything in network_links other than the retired segments
select      network_links_22_2.* 
from 		congestion.network_links_22_2
left join 	changed_seg using (segment_id)
where 		changed_seg is null ;

'''
'''
-- Insert routed results using new map version (23_4) for retired segments

with changed_links AS (
	select link_dir from congestion.network_links_22_2
	except 
	select link_dir from here.routing_streets_23_4)	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_22_2 
	inner join changed_links using (link_dir))
 
 -- insert result to newly created network_links table
INSERT INTO congestion.network_links_23_4
SELECT segment_id, start_vid, end_vid, link_dir, routing.geom, round(st_length(st_transform(geom, 2952))::numeric, 2) as length
FROM   changed_seg
CROSS JOIN LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
				 	   			FROM here.routing_streets_23_4',  -- route using new map version's routing_streets
								start_vid, 
								end_vid)
INNER JOIN here.routing_streets_23_4 routing ON id = edge;
''';

### Check to see if all segments are inserted 
Returns nothing! 

In [33]:
sql = '''
select distinct segment_id from congestion.network_links_22_2
except
select distinct segment_id from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,segment_id


### Check to see if both returned the same number of segment_id
Both have 6558 segments. 

In [34]:
sql = '''

select count(distinct segment_id) from congestion.network_links_22_2
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6558


In [35]:
sql = '''
select count(distinct segment_id) from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6558


## Find if there are new traffic signals to add in 
25?!?!
Check them out on QGIS and see if they are legit. 
Checked: They are mostly legit other than one that are at a midblock and seems like a peds crossing.

In [36]:
sql = '''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

select count(distinct px)
from congestion.network_segments seg
join new_signal on ST_intersects(new_signal.geom, seg.geom)
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,25


## Find closest nodes for these new traffic signals 

Checked on QGIS, mostly look good in distance difference, except for one that has 40m difference (aka the one that is a ped cross). 

In [22]:
'''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

, new_px AS (
	select distinct px, new_signal.geom as px_geom
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))

-- Find new nodes for outdated nodes using nearest neighbour
			
select 	node_id, px, geom, px_geom, dist
from 	here.routing_nodes_23_4
CROSS JOIN LATERAL (SELECT z.px,
							px_geom, 
							(ST_transform(geom, 2952) <-> ST_Transform(z.px_geom, 2952)) as dist	
					FROM new_px z
					ORDER BY (geom <-> z.px_geom)
					LIMIT 1) nodes;
''';

### Exclude traffic signals that should not be included

px 2617 - too far away, nearest node is 40m away

px 2650 - its on a ramp, not a part of network segments

## Route segments that was affected by new traffic signal

This query gathers all new nodes (find by nearest neighbour with traffic signal), as well as the start and end nodes of those affected segments (segments where new traffic signal lies). This creates a new table so its easier to QC on QGIS and make modifications.

```sql
--drop table 
CREATE TABLE congestion.network_segments_link_temp AS 
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom as px_geom
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px) -- where they are not already in old version
	where activationdate >= '2022-04-17' and a.px is null 	
        and px != '2617' -- too far away
    	and px != '2650') -- a ramp
-- px that are new
, new_px AS (
	select distinct px, px_geom
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))
-- segments affected by new signals
, affected_segment AS (
	select distinct start_vid, end_vid, segment_id
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))
, affected_links AS (
	select link_dir 
	from affected_segment
	inner join congestion.network_links_23_4 USING (segment_id)
)
-- nodes that are closest to new traffic signal
, new_nodes AS (
	select 	node_id::int as node_id
	from 	 new_px
	CROSS JOIN LATERAL (SELECT node_id,
								geom, 
								(ST_transform(geom, 2952) <-> ST_Transform(px_geom, 2952)) as dist	
						FROM here.routing_nodes_23_4
						ORDER BY (geom <-> px_geom)
						LIMIT 1) nodes)
-- nodes for re-routing
, nodes AS (
	SELECT array_agg(node_id::int) as nodes_to_route 
	-- aggregate them into one array for many-to-many routing
	FROM	(select node_id 
			 FROM here.routing_nodes_23_4
			 where node_id in (30451170,852108714,30584642)
			 union 
			 select start_vid as node_id --start vid of retired segments
			 FROM affected_segment 
			 union 
			 select end_vid as node_id--end vid of retired segments
			 FROM affected_segment 
			 union
			 select distinct node_id 
			 FROM  new_nodes  
			 )a)
			
, results AS(
	SELECT results.*, link_dir, routing_grid.geom
	FROM nodes
	, LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
						   FROM here.routing_streets_23_4',
				nodes_to_route, nodes_to_route) results
	INNER JOIN here.routing_streets_23_4 routing_grid ON id = edge)	
, cleaned_results AS (
	SELECT 7113+row_number() over () as segment_id, 
				start_vid, 
				end_vid, 
				array_agg(link_dir order by path_seq) as link_set, 
				st_linemerge(st_union(s.geom)) as geom, 
				sum(cost) as length

	FROM results s
	LEFT OUTER JOIN (SELECT node_id FROM congestion.network_nodes_23
					 UNION 
					 SELECT unnest(nodes_to_route) FROM  nodes) network_nodes_23
					 ON node = node_id AND node != start_vid
	GROUP BY start_vid, end_vid
	HAVING COUNT(node_id) = 0 -- exclude routed results that went any nodes 
	order by start_vid, end_vid	)
, cleaned_links AS (
	select 	
		segment_id, 
		start_vid, 
		end_vid,
		link_dir, 
		geom, 
		round(ST_length(st_transform(geom, 2952))::numeric,2) as length,
		case when affected.link_dir is null then 'F' else 'T' end as used 
	from (select segment_id, start_vid, end_vid, unnest(link_set) as link_dir
		  from cleaned_results) a
	inner join here.routing_streets_23_4 using (link_dir)
	left join affected_links affected using (link_dir)
	order by segment_id)
	
select segment_id, start_vid, end_vid, ST_linemerge(ST_union(geom)) , sum(length), false, gis.direction_from_line(ST_linemerge(ST_union(geom))) as dir
from  cleaned_links
group by segment_id, start_vid, end_vid
having 'F' != ALL(array_agg(used));
```

### Make new links after QC into a temp table

In [ ]:
'''
insert into congestion.network_links_23_4
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom as px_geom
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px) -- where they are not already in old version
	where activationdate >= '2022-04-17' and a.px is null 	
        and px != '2617' -- too far away
    	and px != '2650') -- a ramp
-- px that are new
, new_px AS (
	select distinct px, px_geom
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))
-- segments affected by new signals
, affected_segment AS (
	select distinct start_vid, end_vid, segment_id
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))
, affected_links AS (
	select link_dir 
	from affected_segment
	inner join congestion.network_links_23_4 USING (segment_id)
)
-- nodes that are closest to new traffic signal
, new_nodes AS (
	select 	node_id::int as node_id
	from 	 new_px
	CROSS JOIN LATERAL (SELECT node_id,
								geom, 
								(ST_transform(geom, 2952) <-> ST_Transform(px_geom, 2952)) as dist	
						FROM here.routing_nodes_23_4
						ORDER BY (geom <-> px_geom)
						LIMIT 1) nodes)
-- nodes for re-routing
, nodes AS (
	SELECT array_agg(node_id::int) as nodes_to_route 
	-- aggregate them into one array for many-to-many routing
	FROM	(select node_id 
			 FROM here.routing_nodes_23_4
			 where node_id in (30451170,852108714,30584642)
			 union 
			 select start_vid as node_id --start vid of retired segments
			 FROM affected_segment 
			 union 
			 select end_vid as node_id--end vid of retired segments
			 FROM affected_segment 
			 union
			 select distinct node_id 
			 FROM  new_nodes  
			 )a)
			
, results AS(
	SELECT results.*, link_dir, routing_grid.geom
	FROM nodes
	, LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
						   FROM here.routing_streets_23_4',
				nodes_to_route, nodes_to_route) results
	INNER JOIN here.routing_streets_23_4 routing_grid ON id = edge)	
, cleaned_results AS (
	SELECT row_number() over () as segment_id, 
				start_vid, 
				end_vid, 
				array_agg(link_dir order by path_seq) as link_set, 
				st_linemerge(st_union(s.geom)) as geom, 
				sum(cost) as length

	FROM results s
	LEFT OUTER JOIN (SELECT node_id FROM congestion.network_nodes_23
					 UNION 
					 SELECT unnest(nodes_to_route) FROM  nodes) network_nodes_23
					 ON node = node_id AND node != start_vid
	GROUP BY start_vid, end_vid
	HAVING COUNT(node_id) = 0 -- exclude routed results that went any nodes 
	order by start_vid, end_vid	)
, cleaned_links AS (
	select 	
		segment_id, 
		start_vid, 
		end_vid,
		link_dir, 
		geom, 
		round(ST_length(st_transform(geom, 2952))::numeric,2) as length,
		case when affected.link_dir is null then 'F' else 'T' end as used 
	from (select segment_id, start_vid, end_vid, unnest(link_set) as link_dir
		  from cleaned_results) a
	inner join here.routing_streets_23_4 using (link_dir)
	left join affected_links affected using (link_dir)
	order by segment_id)
	
, new_seg AS (
	select 7113+row_number() over() as segment_id, start_vid, end_vid, 
	array_agg(link_dir) as links, 
	ST_linemerge(ST_union(geom)), 
	sum(length), 
	false, 
	gis.direction_from_line(ST_linemerge(ST_union(geom))) as dir
from  cleaned_links
group by segment_id, start_vid, end_vid
having 'F' != ALL(array_agg(used)))

select segment_id, start_vid, end_vid, link_dir, geom, length
from (
	select segment_id, start_vid, end_vid, unnest(links) as link_dir
	from new_seg)a
inner join here.routing_streets_23_4 using (link_dir)
order by segment_id
'''

### Add them to new segment table

In [ ]:
'''
insert into  congestion.network_segments_23_4

select segment_id, start_vid, end_vid, 
ST_linemerge(ST_union(geom)) , sum(length), 
false, gis.direction_from_line(ST_linemerge(ST_union(geom))) as dir
from  congestion.network_links_23_4
where segment_id > 7113
group by segment_id, start_vid, end_vid;
'''

In [ ]:
'''insert into congestion.network_nodes_23 
with temp as (
select start_vid as node_id from congestion.network_links_23_4
union 
select end_vid from congestion.network_links_23_4)
select distinct node_id, (ST_dump(a.geom)).geom
from congestion.network_nodes_23
right join temp using (node_id)
inner join here.routing_nodes_23_4 a using (node_id)
where network_nodes_23.node_id is null
'''


In [ ]:
'''
insert into congestion.network_int_px_23_4
with new_signal as (
	select px, traffic_signal.node_id as int_id, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom as px_geom
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px) -- where they are not already in old version
	where activationdate >= '2022-04-17' and a.px is null 	
        and px != '2617' -- too far away
    	and px != '2650') -- a ramp
-- px that are new
, new_px AS (
	select distinct px, px_geom, int_id
	from congestion.network_segments seg
	join new_signal on ST_intersects(new_signal.bgeom, seg.geom))

-- nodes that are closest to new traffic signal
, new_nodes AS (
	select 	node_id::int as node_id, px, int_id
	from 	 new_px
	CROSS JOIN LATERAL (SELECT node_id,
								geom, 
								(ST_transform(geom, 2952) <-> ST_Transform(px_geom, 2952)) as dist	
						FROM here.routing_nodes_23_4
						ORDER BY (geom <-> px_geom)
						LIMIT 1) nodes)
, temp as (
select start_vid as node_id from congestion.network_links_23_4
union 
select end_vid from congestion.network_links_23_4)

, missing_nodes AS (
	select distinct node_id, (ST_dump(a.geom)).geom as node_geom
	from congestion.network_int_px_22_2
	right join temp using (node_id)
	inner join here.routing_nodes_23_4 a using (node_id)
	where network_int_px_22_2.node_id is null)
select node_id, case when int_id = 0 then null else int_id end as int_id, px, node_geom, int.geom as ints_geom, null as dist
from missing_nodes
left join new_nodes using (node_id)
left join gis_core.centreline_intersection_point int on intersection_id = int_id
'''

### Retire old segments

In [ ]:
'''
insert into congestion.network_segments_retired
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and activationdate <'2023-12-01' -- date of last version's max activatation date and this verion's max
		and a.px is null  
		and px != '2617' -- too far away
    	and px != '2650') 

select 	segment_id, start_vid, end_vid, seg.geom, total_length, highway, 
		_s.int_id as start_int, _t.int_id as end_int, 
		_s.px::int start_px, _t.px::int as end_px, 
		'22_2' as here_version, '20231001' as centreline_version, 
		'2024-01-02'::date as retired_date, 'new traffic signal' as retired_reason, 
	  	 null::int[] as replaced_id, -- to be updated later during re-routing new segments
		min(dt) as valid_from, max(dt) as valid_to

from congestion.network_segments seg

JOIN new_signal on ST_intersects(new_signal.geom, seg.geom) -- where segments intersects with new traffic signals 
INNER JOIN congestion.network_segments_daily USING (segment_id) -- to get valid from to date ranges
INNER JOIN congestion.network_int_px_22_2 _s on start_vid = _s.node_id -- to get equivalent start px and int_id
INNER JOIN congestion.network_int_px_22_2 _t on end_vid  = _t.node_id -- to get equivalent end px and int_id
WHERE segment_id  <= 7113 and segment_id not in (3573,170,3951,778)
GROUP BY segment_id, start_vid, end_vid, seg.geom, total_length, highway, start_int, end_int, start_px, end_px, here_version, centreline_version, 
		retired_date, retired_reason, replaced_id;
'''

'''
update congestion.network_segments_retired
set replaced_id = replaced
from 
	(select segment_id, array_agg(s) as replaced 
	 from (
		select distinct seg.segment_id s, retired.segment_id , seg.start_vid, seg.end_vid, retired.start_vid, retired.end_vid, seg.geom, retired.geom
		from congestion.network_segments_retired retired 
		inner join congestion.network_segments_23_4 seg on ST_intersects(retired.geom, seg.geom)
		where 
			retired_reason = 'new traffic signal' and 
		 	seg.segment_id > 7113 and 
		  	gis.direction_from_line(retired.geom)  = direction)a
	 group by segment_id)b
where b.segment_id = network_segments_retired.segment_id		
'''

23_4 version.

Major changes: 
46 old segments retired. 

55 segments retired. 

# Validate

## Make sure there are no duplication 

### Table `network_segments`

Total number of rows in network_segments

In [79]:
sql = '''
select count(1) 
from congestion.network_segments_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6582


No duplicated segment_id

In [80]:
sql = '''
select count(1) 
from (select distinct segment_id from congestion.network_segments_23_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6582


No duplicated sets of the same segment

In [82]:
sql = '''
select count(1) 
from (select distinct start_vid, end_vid from congestion.network_segments_23_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6582


### Table `network_nodes`

Total number of nodes

In [87]:
sql = '''
select count(1) 
from congestion.network_nodes_23
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,3267


There are no duplicated node_ids

In [88]:
sql = '''
select count(1) 
from (select distinct node_id from congestion.network_nodes_23)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,3267


Are there nodes in network_segments_23 that are not in nodes table?

Returns none.

In [89]:
sql = '''
WITH all_nodes AS (
	select start_vid as node_id
	from congestion.network_segments_23_4
	union 
	select end_vid as node_id
	from congestion.network_segments_23_4)
select node_id from all_nodes
except 
select node_id from congestion.network_nodes_23
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,node_id


Are all the nodes in here referenced in the new px<>int<>node table? 

Returned 2, 30375021 and 1050298501. Checked in the segments table, they were not even used, deleting.

In [91]:
sql = '''
select node_id::int from congestion.network_nodes_23
except 
select node_id::int from congestion.network_int_px_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,node_id
0,1050298501
1,30375021


### Table `network_links`

Total number of links

In [111]:
sql = '''

select count(1) 
from congestion.network_links_23_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,50476


a bunch of them are duplicate, this is where I remembered I did not get rid of the old segments yet

In [112]:
sql = '''
select count(1) 
from (select distinct link_dir from congestion.network_links_23_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,50476


#### 2025-04-25 Here I am a year later, redoing this. After knowing link_dirs and node_ids can stay the same but have different geometries. We need to update the geoms.

They are not that different or far away, just a small shift.

In [3]:
sql = '''
select node_id, og.geom, v.geom, ST_Setsrid(og.geom, 2952) <-> ST_Setsrid(v.geom, 2952) as dist
from congestion.network_nodes_23 og
left join (select distinct node_id, geom from here.routing_nodes_23_4) v using (node_id)
order by abs(ST_Setsrid(og.geom, 2952) <-> ST_Setsrid(v.geom, 2952)) desc
'''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head(5)  

,node_id,geom,geom,dist
0,30420392.0,0101000020E61000000EDB166536D653C09E0C8E9257D3...,None,NaN
1,30420390.0,0101000020E6100000A3586E6935D653C0815B77F354D3...,None,NaN
2,30445532.0,0101000020E61000006891ED7C3FD353C06631B1F9B8E2...,0104000020E6100000010000000101000000378E588B4F...,0.001007
3,30334294.0,0101000020E61000007311DF8959E353C02CF180B229CF...,0104000020E610000001000000010100000041F163CC5D...,0.000626
4,30334658.0,0101000020E6100000F60B76C3B6E353C0B8E9CF7EA4D0...,0104000020E6100000010000000101000000CF143AAFB1...,0.000623


Have to update geometry field in the zlevels to make multipoint into single point 

```sql
update congestion.network_nodes_23
set geom = a.geom
from (select distinct node_id, geom from here.routing_nodes_23_4)a
where a.node_id = network_nodes_23.node_id
```

Have to do the same thing for links. For links, there are 1574/50482 link_dirs that had a different length, with max length change 0.002m which is so little.

In [4]:
sql = '''
select count(1), max(ST_length(ST_Setsrid(og.geom, 2952)) - ST_length(ST_Setsrid(v.geom, 2952)))
from congestion.network_links_23_4 og
left join (select distinct link_dir, geom from here.routing_streets_23_4) v using (link_dir)
where ST_length(ST_Setsrid(og.geom, 2952)) - ST_length(ST_Setsrid(v.geom, 2952)) >0
'''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count,max
0,1574,0.001868


```sql
update congestion.network_links_23_4
set geom = a.geom
from (select distinct link_dir, geom from here.routing_streets_23_4)a
where a.link_dir = network_links_23_4.link_dir
```

We will then have to recreate the segments using these new links geoms. Done!

Lets validate again

Wanted to make sure I actually retired all the segments that have changed in 23_4, I missed ten. 

In [5]:
sql = '''with temp as (
select segment_id from congestion.network_segments 
except 
select segment_id from congestion.network_segments_23_4_geom )

select segment_id from temp
except 
select segment_id from congestion.network_segments_retired
where here_version = '22_2'
'''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head(10)  

,segment_id
0,680
1,5496
2,3588
3,6213
4,5602
5,3589
6,5760
7,498
8,2328
9,5757
